In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pyodbc
import math

In [ ]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_dincident,inc_time,inc_dreported,inc_submittedtime,inc_reportedby from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC' and inc_organisation='QA'"
df = pd.read_sql(query, sql_conn)
print(df)

In [ ]:
df['inc_time'] = df['inc_time'].str.replace(':','')
df['inc_submittedtime'] = df['inc_submittedtime'].str.replace(':','')

In [ ]:
df_inc = df.copy()
df_inc = df_inc[df_inc.inc_dincident > '2016-03-31']
df_inc = df_inc[df_inc.inc_dincident < '2019-11-18']
print(df_inc)

In [ ]:
df_inc['hour'] = df_inc['inc_time'].str[:2]
df_inc['minute'] = df_inc['inc_time'].str[2:]
df_inc = df_inc.astype({'hour':'int'})
df_inc = df_inc.astype({'minute':'int'})
df_inc = df_inc.astype({'inc_time':'int'})
print(df_inc[df_inc.hour == 23])

In [ ]:
total_days = (max(df_inc.inc_dincident)-min(df_inc.inc_dincident)).days

In [ ]:
f_first = lambda i: i*100
f_second = lambda i: i*100+30
bins = [f(i) for i in range(25) for f in (f_first,f_second)]
bins = bins[:-1]
print(bins)
data_x = bins[:-1]
data_y_tot,_ = np.histogram(df_inc['inc_time'].values, bins=bins)
data_y = data_y_tot/total_days
data_y_err = np.sqrt(data_y_tot)/total_days

In [ ]:
# Compute pie slices
N = 48
width = 2 * np.pi/N
theta = np.linspace(0.0 + width, 2 * np.pi + width, N, endpoint=False)
theta_deg = np.linspace(0.0 + 360/(2*N), 360 + 360/(2*N), N, endpoint=False)
radii = data_y
radii_err = data_y_err

ax = plt.subplot(111, projection='polar')
ax.bar(theta, radii, width=width,label='Data');
ax.bar(theta, radii_err, bottom=radii-radii_err/2,width=width,alpha=0.5,color='none',hatch="////",label='Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
plt.xlabel('hour in day')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.04,0))
lines, labels = plt.thetagrids( theta_deg, (data_x) )

In [ ]:
morning_radii = np.zeros(len(radii))
afternoon_radii = np.zeros(len(radii))
evening_radii = np.zeros(len(radii))
for i in range(24):
    print(i)
    if i <= 7: morning_radii[i] = radii[i]
    elif i >= 8 and i <= 15: afternoon_radii[i] = radii[i]
    else: evening_radii[i] = radii[i]

In [ ]:
ax = plt.subplot(111, projection='polar');
ax.bar(theta, morning_radii, width=width,label='night');
ax.bar(theta, afternoon_radii, width=width,label='daytime');
ax.bar(theta, evening_radii, width=width,label='evening');
ax.bar(theta, 2*radii_err, bottom=radii-radii_err,width=width,alpha=0.5,color='none',hatch="////",label='Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
#ax.set_yticklabels([])
plt.xlabel('hour occured')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.04,0))
lines, labels = plt.thetagrids( theta_deg, (data_x) );
ax.axvspan(theta[np.argmax(radii)]-width/2,theta[np.argmax(radii)]+width/2,color='red',alpha=0.3);

In [ ]:
reportedby_counts = df['inc_reportedby'].value_counts().drop(labels=[''])
reportedby_counts_nlargest = reportedby_counts.nlargest(9)
thresh = reportedby_counts_nlargest[-1]
reportedby_counts = reportedby_counts.append(pd.Series(reportedby_counts[reportedby_counts.values < thresh].sum(),index=['Other']))
reportedby_counts = reportedby_counts[reportedby_counts.values >= thresh]

ax = plt.subplot(111, projection='polar')
cumulative_bottom = 0
for i,counts in reportedby_counts.items():
    radii_reported_tot,_ = np.histogram(df_inc[df_inc['inc_reportedby']==i].hour.values, bins=bins)
    radii_reported = radii_reported_tot/total_days
    ax.bar(theta, radii_reported, bottom=cumulative_bottom, width=width, label=i);
    cumulative_bottom += radii_reported

ax.bar(theta, radii_err, bottom=cumulative_bottom-radii_err/2,width=width,alpha=0.5,color='none',hatch="////",label='Total Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
#ax.set_yticklabels([])
plt.xlabel('hour occured')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.1,0),title='Reported by:')
lines, labels = plt.thetagrids( theta_deg, (data_x) )

In [ ]:
ax = plt.subplot(111, projection='polar')
ax.bar(theta, radii, width=width,label='Data');
ax.bar(theta, radii_err, bottom=radii-radii_err/2,width=width,alpha=0.5,color='none',hatch="////",label='Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
#ax.set_yticklabels([])
plt.title('hour occured',y=1.1)
lines, labels = plt.thetagrids( theta_deg, (data_x) )
ax.axvline(theta[7]-width/2,color='orange');
ax.axvline(theta[15]-width/2,color='orange');
ax.axvline(theta[12],color='green');
ax.axvline(theta[20],color='green');
ax.axvline(theta[20]-width/2,color='red');
ax.axvline(theta[8]-width/2,color='red');
ax.annotate('Early shift start',xy=(theta[7]-width/2,ax.get_ylim()[1]),xytext=(theta[7]-width/2,1.1),color='orange',ha='center',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Early shift end',xy=(theta[15]-width/2,ax.get_ylim()[1]),xytext=(theta[15]-width/2,0.9),color='orange',ha='right',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Late shift start',xy=(theta[12],ax.get_ylim()[1]),xytext=(theta[12],0.9),color='green',ha='center',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Late shift end',xy=(theta[20],ax.get_ylim()[1]),xytext=(theta[20],0.9),color='green',ha='right',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Night shift start',xy=(theta[20]-width/2,ax.get_ylim()[1]),xytext=(theta[20]-width/2,0.9),color='red',ha='right',
           arrowprops=dict(facecolor='red',alpha=0.1));
ax.annotate('Night shift end',xy=(theta[8]-width/2,ax.get_ylim()[1]),xytext=(theta[8]-width/2,0.9),color='red',ha='left',
           arrowprops=dict(facecolor='red',alpha=0.1));

In [ ]:
early_radii = np.zeros(len(radii))
late_radii = np.zeros(len(radii))
night_radii = np.zeros(len(radii))
for i in range(24):
    if i >= 8 and i <= 14: early_radii[i] = radii[i]
    elif i >= 15 and i <= 20: late_radii[i] = radii[i]
    else: night_radii[i] = radii[i]
ax = plt.subplot(111, projection='polar');
ax.bar(theta, early_radii, width=width,label='early',color='orange');
ax.bar(theta, late_radii, width=width,label='late',color='green');
ax.bar(theta, night_radii, width=width,label='night',color='blue');
ax.bar(theta, 2*radii_err, bottom=radii-radii_err,width=width,alpha=0.5,color='none',hatch="////",label='uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
ax.set_yticklabels(['',0.2,'',0.4,'',0.6])
#plt.xlabel('hour occured')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.04,0.8))
lines, labels = plt.thetagrids( theta_deg, (data_x) );
ax.axvspan(theta[np.argmax(radii)]-width/2,theta[np.argmax(radii)]+width/2,color='red',alpha=0.3);
ax.axvline(theta[7]-width/2,color='orange');
ax.axvline(theta[15]-width/2,color='orange');
ax.axvline(theta[12],color='green');
ax.axvline(theta[20],color='green');
ax.axvline(theta[20]-width/2,color='blue');
ax.axvline(theta[8]-width/2,color='blue');
ax.annotate('Early shift start',xy=(theta[7]-width/2,ax.get_ylim()[1]),xytext=(theta[7]-width/2,1.1),color='orange',ha='center',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Early shift end',xy=(theta[15]-width/2,ax.get_ylim()[1]),xytext=(theta[15]-width/2,0.9),color='orange',ha='right',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Late shift start',xy=(theta[12],ax.get_ylim()[1]),xytext=(theta[12],0.9),color='green',ha='center',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Late shift end',xy=(theta[20],ax.get_ylim()[1]),xytext=(theta[20],0.9),color='green',ha='right',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Night shift start',xy=(theta[20]-width/2,ax.get_ylim()[1]),xytext=(theta[20]-width/2,0.9),color='blue',ha='right',
           arrowprops=dict(facecolor='blue',alpha=0.1));
ax.annotate('Night shift end',xy=(theta[8]-width/2,ax.get_ylim()[1]),xytext=(theta[8]-width/2,0.9),color='blue',ha='left',
           arrowprops=dict(facecolor='blue',alpha=0.1));

In [ ]:
night_radii = np.zeros(len(radii))
night_early_radii = np.zeros(len(radii))
early_radii = np.zeros(len(radii))
early_late_radii = np.zeros(len(radii))
late_radii = np.zeros(len(radii))
late_night_radii = np.zeros(len(radii))
for i in range(24):
    if i==7: night_early_radii[i] = radii[i]
    elif i >= 8 and i <= 12: early_radii[i] = radii[i]
    elif i==13 or i==14: early_late_radii[i] = radii[i]
    elif i >= 15 and i <= 19: late_radii[i] = radii[i]
    elif i==20: late_night_radii[i] = radii[i]
    else: night_radii[i] = radii[i]
ax = plt.subplot(111, projection='polar');
ax.bar(theta, night_radii, width=width,label='night',color='green');
ax.bar(theta, night_early_radii, width=width, label='night-early crossover', color='yellow')
ax.bar(theta, early_radii, width=width,label='early',color='orange');
ax.bar(theta, early_late_radii, width=width, label='early-late crossover', color='red')
ax.bar(theta, late_radii, width=width,label='late',color='purple');
ax.bar(theta, late_night_radii, width=width, label='late-night crossover', color='blue')
ax.bar(theta, 2*radii_err, bottom=radii-radii_err,width=width,alpha=0.5,color='none',hatch="////",label='uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
ax.set_yticklabels(['',0.2,'',0.4,'',0.6])
#plt.xlabel('hour occured')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.07,0.6),title='Shifts')
lines, labels = plt.thetagrids( theta_deg, (data_x) );
ax.axvspan(theta[np.argmax(radii)]-width/2,theta[np.argmax(radii)]+width/2,color='red',alpha=0.3);
ax.axvline(theta[7]-width/2,color='orange');
ax.axvline(theta[15]-width/2,color='orange');
ax.axvline(theta[12],color='purple');
ax.axvline(theta[20],color='purple');
ax.axvline(theta[20]-width/2,color='green');
ax.axvline(theta[8]-width/2,color='green');
ax.annotate('Early shift start',xy=(theta[7]-width/2,ax.get_ylim()[1]),xytext=(theta[7]-width/2,1.1),color='orange',ha='center',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Early shift end',xy=(theta[15]-width/2,ax.get_ylim()[1]),xytext=(theta[15]-width/2,0.9),color='orange',ha='right',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Late shift start',xy=(theta[12],ax.get_ylim()[1]),xytext=(theta[12],0.9),color='purple',ha='center',
           arrowprops=dict(facecolor='purple',alpha=0.1));
ax.annotate('Late shift end',xy=(theta[20],ax.get_ylim()[1]),xytext=(theta[20],0.9),color='purple',ha='right',
           arrowprops=dict(facecolor='purple',alpha=0.1));
ax.annotate('Night shift start',xy=(theta[20]-width/2,ax.get_ylim()[1]),xytext=(theta[20]-width/2,0.9),color='green',ha='right',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Night shift end',xy=(theta[8]-width/2,ax.get_ylim()[1]),xytext=(theta[8]-width/2,0.9),color='green',ha='left',
           arrowprops=dict(facecolor='green',alpha=0.1));

In [ ]:
df_rep = df.copy()
df_rep = df_rep[df_rep.inc_dreported > '2016-03-31']
df_rep = df_rep[df_rep.inc_dreported < '2019-10-01']
df_rep['rep_hour'] = df_rep['inc_submittedtime'].str[:2]
df_rep = df_rep.astype({'rep_hour':'int'})
data_y_tot,_ = np.histogram(df_rep.rep_hour.values, bins=bins)
data_y = data_y_tot/total_days
data_y_err = np.sqrt(data_y_tot)/total_days
radii = data_y
radii_err = data_y_err
morning_radii = np.zeros(len(radii))
afternoon_radii = np.zeros(len(radii))
evening_radii = np.zeros(len(radii))
for i in range(24):
    if i <= 7: morning_radii[i] = radii[i]
    elif i >= 8 and i <= 15: afternoon_radii[i] = radii[i]
    else: evening_radii[i] = radii[i]
ax = plt.subplot(111, projection='polar');
ax.bar(theta, morning_radii, width=width,label='night');
ax.bar(theta, afternoon_radii, width=width,label='daytime');
ax.bar(theta, evening_radii, width=width,label='evening');
ax.bar(theta, 2*radii_err, bottom=radii-radii_err,width=width,alpha=0.5,color='none',hatch="////",label='Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
ax.set_yticklabels([])
plt.xlabel('hour reported')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.04,0))
lines, labels = plt.thetagrids( theta_deg, (data_x) );
ax.axvspan(theta[np.argmax(radii)]-width/2,theta[np.argmax(radii)]+width/2,color='red',alpha=0.3);

In [ ]:
reportedby_counts = df['inc_reportedby'].value_counts().drop(labels=[''])
reportedby_counts_nlargest = reportedby_counts.nlargest(9)
thresh = reportedby_counts_nlargest[-1]
reportedby_counts = reportedby_counts.append(pd.Series(reportedby_counts[reportedby_counts.values < thresh].sum(),index=['Other']))
reportedby_counts = reportedby_counts[reportedby_counts.values >= thresh]

ax = plt.subplot(111, projection='polar')
cumulative_bottom = 0
for i,counts in reportedby_counts.items():
    radii_reported_tot,_ = np.histogram(df_rep[df_rep['inc_reportedby']==i].rep_hour.values, bins=bins)
    radii_reported = radii_reported_tot/total_days
    ax.bar(theta, radii_reported, bottom=cumulative_bottom, width=width, label=i);
    cumulative_bottom += radii_reported

ax.bar(theta, radii_err, bottom=cumulative_bottom-radii_err/2,width=width,alpha=0.5,color='none',hatch="////",label='Total Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
ax.set_yticklabels([])
plt.xlabel('hour reported')
plt.title('Hourly number of patient medicine safety events at QA',y=1.1)
plt.legend(loc=(1.1,0),title='Reported by:')
lines, labels = plt.thetagrids( theta_deg, (data_x) )

In [ ]:
ax = plt.subplot(111, projection='polar')
ax.bar(theta, radii, width=width,label='Data');
ax.bar(theta, radii_err, bottom=radii-radii_err/2,width=width,alpha=0.5,color='none',hatch="////",label='Uncertainty');
ax.set_theta_offset(np.pi/2 + width/2)
ax.set_theta_direction(-1)
ax.set_yticklabels([])
#plt.xlabel('hour in day')
plt.title('hour reported',y=1.1)
#plt.legend(loc=(1.04,0))
lines, labels = plt.thetagrids( theta_deg, (data_x) )
ax.axvline(theta[7]-width/2,color='orange');
ax.axvline(theta[15]-width/2,color='orange');
ax.axvline(theta[12],color='green');
ax.axvline(theta[20],color='green');
ax.axvline(theta[20]-width/2,color='red');
ax.axvline(theta[8]-width/2,color='red');
ax.annotate('Early shift start',xy=(theta[7]-width/2,ax.get_ylim()[1]),xytext=(theta[7]-width/2,1.1),color='orange',ha='center',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Early shift end',xy=(theta[15]-width/2,ax.get_ylim()[1]),xytext=(theta[15]-width/2,0.9),color='orange',ha='right',
           arrowprops=dict(facecolor='orange',alpha=0.1));
ax.annotate('Late shift start',xy=(theta[12],ax.get_ylim()[1]),xytext=(theta[12],0.9),color='green',ha='center',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Late shift end',xy=(theta[20],ax.get_ylim()[1]),xytext=(theta[20],0.9),color='green',ha='right',
           arrowprops=dict(facecolor='green',alpha=0.1));
ax.annotate('Night shift start',xy=(theta[20]-width/2,ax.get_ylim()[1]),xytext=(theta[20]-width/2,0.9),color='red',ha='right',
           arrowprops=dict(facecolor='red',alpha=0.1));
ax.annotate('Night shift end',xy=(theta[8]-width/2,ax.get_ylim()[1]),xytext=(theta[8]-width/2,0.9),color='red',ha='left',
           arrowprops=dict(facecolor='red',alpha=0.1));